In [1]:
###Imports
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.transforms as transforms
from torchvision import models
from torchvision import datasets
from torch.utils.data import DataLoader

import os
import ntpath
import cv2
from cv2 import imread, resize, imshow # manipulating the images

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# from data import loaders
# from model import Classifier_net
# from check import train, test

/raid/home/ujjawald/miniconda3/envs/ujju/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Augumentation 

In [2]:
# Melanoma x 4
# Seborrheic_keratosis x 6

def produce_new_img(img2: cv2) -> tuple:
    """
    function to reproduse a new manipulated (rotating of flipping the original one)
    image from the read one, To increase the dimension of the dataset, avoiding overfitting of a single class.

    Args:
        img2 (cv2): the read image from cv2 module.

    Returns:
        new_images (tuple): a tuple of the new manipulated images.
    """
    imga = cv2.rotate(img2, cv2.ROTATE_90_CLOCKWISE)
    imgb = cv2.rotate(img2, cv2.ROTATE_90_COUNTERCLOCKWISE)
    imgc = cv2.rotate(img2, cv2.ROTATE_180)
    imgd = cv2.flip(img2, 0)
    imge = cv2.flip(img2, 1)
    new_imges = imga, imgb, imgc, imgd ,imge
#     new_imges = imgc, imgd ,imge
    return new_imges

"""
train_melanoma = "./data/train/melanoma"
for c,filename in enumerate(os.scandir(train_melanoma)):
    if filename.is_file():
        file_to_read = train_melanoma + "/" + ntpath.basename(filename)
        img = imread(file_to_read)
#         img = mpimg.imread(file_to_read)
#         imgplot = plt.imshow(img)
#         plt.show()
        new_img = produce_new_img(img)
        for i,aug_img in enumerate(new_img):
            cv2.imwrite(file_to_read + "_aug_" + str(i) + ".jpg",aug_img)
#             print("->",i)
#         print(c)
"""
        
"""
train_seborrheic_keratosis = "./data/train/seborrheic_keratosis"
for c,filename in enumerate(os.scandir(train_seborrheic_keratosis)):
    if filename.is_file():
        file_to_read = train_seborrheic_keratosis + "/" + ntpath.basename(filename)
        img = imread(file_to_read)
#         img = mpimg.imread(file_to_read)
#         imgplot = plt.imshow(img)
#         plt.show()
        new_img = produce_new_img(img)
        for i,aug_img in enumerate(new_img):
            cv2.imwrite(file_to_read + "_aug_" + str(i) + ".jpg",aug_img)
#             print("->",i)
#         print(c)
"""

'\ntrain_seborrheic_keratosis = "./data/train/seborrheic_keratosis"\nfor c,filename in enumerate(os.scandir(train_seborrheic_keratosis)):\n    if filename.is_file():\n        file_to_read = train_seborrheic_keratosis + "/" + ntpath.basename(filename)\n        img = imread(file_to_read)\n#         img = mpimg.imread(file_to_read)\n#         imgplot = plt.imshow(img)\n#         plt.show()\n        new_img = produce_new_img(img)\n        for i,aug_img in enumerate(new_img):\n            cv2.imwrite(file_to_read + "_aug_" + str(i) + ".jpg",aug_img)\n#             print("->",i)\n#         print(c)\n'

### model.py

In [3]:
### Importing the model (inception v3 CNN) pretrained on Imagenet
Classifier_net = models.inception_v3()

# Note: Modify final fully connected network by using nn.Sequential() to define layers

Classifier_net.fc = nn.Sequential(nn.Linear(2048, 512),
				nn.ReLU(),
				nn.Dropout(p=0.25),
				nn.Linear(512, 64),
				nn.ReLU(),
				nn.Dropout(p=0.25),
				nn.Linear(64, 3))

/raid/home/ujjawald/miniconda3/envs/ujju/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


### data.py

In [4]:
# training set
train_transforms = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
        ])

# test and valid sets
check_transforms = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
        ])

### Loading files as a dataset
trainset = datasets.ImageFolder('./data/train/', transform=train_transforms)
testset = datasets.ImageFolder('./data/testData/', transform=check_transforms)
validset = datasets.ImageFolder('./data/valid/', transform=check_transforms)


print(trainset)
print(testset)
print(validset)

### Creating batches of iterables from the datasets
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=True)
validloader = DataLoader(validset, batch_size=32, shuffle=True)

loaders = {'train':trainloader, 'test':testloader, 'valid':validloader}

Dataset ImageFolder
    Number of datapoints: 4392
    Root location: ./data/train/
    StandardTransform
Transform: Compose(
               Resize(size=(299, 299), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 600
    Root location: ./data/testData/
    StandardTransform
Transform: Compose(
               Resize(size=(299, 299), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 150
    Root location: ./data/valid/
    StandardTransform
Transform: Compose(
             

### check.py

In [5]:
###Imports
import numpy as np
import torch
from sklearn.metrics import confusion_matrix

#### Methods for Threat Score
def getCM(true_y, pred_y):
    true_y = true_y
    pred_y = pred_y
    return confusion_matrix(true_y, pred_y,labels=[0,1,2])

def getTS(tp,tn,fp,fn):
    return tp/(tp+fp+fn)

def calcTS(true_y, pred_y):
    
    confusion_metric = (getCM(true_y,pred_y)).T
    
    mel_TP = confusion_metric[0][0]
    mel_TN = confusion_metric[1][1] + confusion_metric[1][2] + confusion_metric[2][1] + confusion_metric[2][2]
    mel_FP = confusion_metric[0][1] + confusion_metric[0][2] 
    mel_FN = confusion_metric[1][0] + confusion_metric[2][0]
  
    nev_TP = confusion_metric[1][1]
    nev_TN = confusion_metric[0][0] + confusion_metric[2][0] + confusion_metric[0][2] + confusion_metric[2][2]
    nev_FP = confusion_metric[1][0] + confusion_metric[1][2] 
    nev_FN = confusion_metric[0][1] + confusion_metric[2][1]

    sk_TP = confusion_metric[2][2]
    sk_TN = confusion_metric[0][0] + confusion_metric[0][1] + confusion_metric[1][0] + confusion_metric[1][1]
    sk_FP = confusion_metric[2][0] + confusion_metric[2][1] 
    sk_FN = confusion_metric[0][2] + confusion_metric[1][2]
    
    mel_TS = getTS(mel_TP,mel_TN,mel_FP,mel_FN)
    nev_TS = getTS(nev_TP,nev_TN,nev_FP,nev_FN)
    sk_TS = getTS(sk_TP,sk_TN,sk_FP,sk_FN)
    
    return (mel_TS + nev_TS + sk_TS)/3

### Training method
def train(epochs, loaders, model, optimizer, criterion, device, save_path, verbose=True,use_saved_model=False):
    
    if(use_saved_model == True):
        saved_model = torch.load(save_path)
        saved_model.eval()
        print("Saved Model Returned")
        return saved_model
      
    ### Moving model to hardware accelerator
    model.to(device)

    valid_loss_min = 10 #Lowest Achieved.

    print("Training starts...\n")

    ### Training and Validation Loop
    for e in range(epochs):
        train_loss = 0.0
        valid_loss = 0.0
        print("Epochs Number:", e+1)

        # Train loop
        n_bactches = len(loaders['train'])
        model.train()
        for batch_idx, (feature, label) in enumerate(loaders['train']):
            feature, label = feature.to(device), label.to(device)
            print("\tTrain Batch Number: [", batch_idx,", ",n_bactches,"]")
            optimizer.zero_grad()

            log_ps, aux_outputs = model(feature)
            loss1 = criterion(log_ps, label)
            loss2 = criterion(aux_outputs, label)
            loss = loss1 + 0.4 * loss2

            loss.backward()
            optimizer.step()

            train_loss += (1 / (batch_idx + 1)) * (loss.item() - train_loss)
        
        # Validation loop
        n_bactches = len(loaders['valid'])
        model.eval()
        for batch_idx, (feature, label) in enumerate(loaders['valid']):
            feature, label = feature.to(device), label.to(device)
            print("\tValidation Batch Number: [", batch_idx,", ",n_bactches,"]")

            log_ps = model(feature)
            loss = criterion(log_ps, label)

            valid_loss += (1 / (batch_idx + 1)) * (loss.item() - valid_loss)

        # Print out results
        if verbose:
            print('Epoch: {} \tTraining Loss: {} \tValidation Loss: {}'.format(e + 1, train_loss, valid_loss))
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            print('Saving Model...')
    
    print("Training ends...\n")
    return model

### Testing method
def test(loaders, model, criterion, device, verbose=True):

    ### Moving model to hardware accelerator
    model.to(device)

    # Initializing test results
    test_loss = 0.0
    correct = 0
    total = 0
    
    true_labels = []
    pred_labels = []
    
    print("Testing starts...\n")
    # Test loop
    model.eval()
    for batch_idx, (feature, label) in enumerate(loaders['test']):
        feature, label = feature.to(device), label.to(device)

        log_ps = model(feature)
        loss = criterion(log_ps, label)

        test_loss += (1 / (batch_idx + 1)) * (loss.item() - test_loss)

        pred = log_ps.data.max(1, keepdim=True)[1]
        correct += np.sum((label.t()[0] == pred).cpu().numpy())
        total += label.shape[0]
        
        true_labels.extend(label.tolist())
        pred_labels.extend(pred)

    pred_la = [i.detach().cpu().numpy().tolist()[0] for i in pred_labels]
    
    threat_score = calcTS(true_labels, pred_la)
    
    if verbose:
        print('Test Loss: {}\n'.format(test_loss))
        print('Test Accuracy: {}%, ({}/{})'.format(100 * correct / total, correct, total))
        print('Threat Score: {}%'.format(threat_score))

    print("Testing ends...\n")

"""
Threat Score =  TP/(TP+FN+FP)
"""

'\nThreat Score =  TP/(TP+FN+FP)\n'

### main.py

In [6]:
### Tunable Hyperparameters (to change fully connected layer, modify model.py)
lr = 0.0001
epochs = 20

### Optimizer and Loss definition
model_criterion = nn.CrossEntropyLoss()
model_optimizer = optim.SGD(Classifier_net.parameters(), lr)

### checking for hardware accelarators
accelerator = 'cuda' if torch.cuda.is_available() else 'cpu'
# accelerator = 'cpu'
print("Device =", accelerator)

### begin training loop
Classifier_net = train(epochs, loaders, Classifier_net, model_optimizer, model_criterion, accelerator, './Classifier_net.pt', verbose=True)

Device = cuda
Training starts...

Epochs Number: 1
	Train Batch Number: [ 0 ,  138 ]
	Train Batch Number: [ 1 ,  138 ]
	Train Batch Number: [ 2 ,  138 ]
	Train Batch Number: [ 3 ,  138 ]
	Train Batch Number: [ 4 ,  138 ]
	Train Batch Number: [ 5 ,  138 ]
	Train Batch Number: [ 6 ,  138 ]
	Train Batch Number: [ 7 ,  138 ]
	Train Batch Number: [ 8 ,  138 ]
	Train Batch Number: [ 9 ,  138 ]
	Train Batch Number: [ 10 ,  138 ]
	Train Batch Number: [ 11 ,  138 ]
	Train Batch Number: [ 12 ,  138 ]
	Train Batch Number: [ 13 ,  138 ]
	Train Batch Number: [ 14 ,  138 ]
	Train Batch Number: [ 15 ,  138 ]
	Train Batch Number: [ 16 ,  138 ]
	Train Batch Number: [ 17 ,  138 ]
	Train Batch Number: [ 18 ,  138 ]
	Train Batch Number: [ 19 ,  138 ]
	Train Batch Number: [ 20 ,  138 ]
	Train Batch Number: [ 21 ,  138 ]
	Train Batch Number: [ 22 ,  138 ]
	Train Batch Number: [ 23 ,  138 ]
	Train Batch Number: [ 24 ,  138 ]
	Train Batch Number: [ 25 ,  138 ]
	Train Batch Number: [ 26 ,  138 ]
	Train Batch N

	Train Batch Number: [ 86 ,  138 ]
	Train Batch Number: [ 87 ,  138 ]
	Train Batch Number: [ 88 ,  138 ]
	Train Batch Number: [ 89 ,  138 ]
	Train Batch Number: [ 90 ,  138 ]
	Train Batch Number: [ 91 ,  138 ]
	Train Batch Number: [ 92 ,  138 ]
	Train Batch Number: [ 93 ,  138 ]
	Train Batch Number: [ 94 ,  138 ]
	Train Batch Number: [ 95 ,  138 ]
	Train Batch Number: [ 96 ,  138 ]
	Train Batch Number: [ 97 ,  138 ]
	Train Batch Number: [ 98 ,  138 ]
	Train Batch Number: [ 99 ,  138 ]
	Train Batch Number: [ 100 ,  138 ]
	Train Batch Number: [ 101 ,  138 ]
	Train Batch Number: [ 102 ,  138 ]
	Train Batch Number: [ 103 ,  138 ]
	Train Batch Number: [ 104 ,  138 ]
	Train Batch Number: [ 105 ,  138 ]
	Train Batch Number: [ 106 ,  138 ]
	Train Batch Number: [ 107 ,  138 ]
	Train Batch Number: [ 108 ,  138 ]
	Train Batch Number: [ 109 ,  138 ]
	Train Batch Number: [ 110 ,  138 ]
	Train Batch Number: [ 111 ,  138 ]
	Train Batch Number: [ 112 ,  138 ]
	Train Batch Number: [ 113 ,  138 ]
	Train

	Train Batch Number: [ 26 ,  138 ]
	Train Batch Number: [ 27 ,  138 ]
	Train Batch Number: [ 28 ,  138 ]
	Train Batch Number: [ 29 ,  138 ]
	Train Batch Number: [ 30 ,  138 ]
	Train Batch Number: [ 31 ,  138 ]
	Train Batch Number: [ 32 ,  138 ]
	Train Batch Number: [ 33 ,  138 ]
	Train Batch Number: [ 34 ,  138 ]
	Train Batch Number: [ 35 ,  138 ]
	Train Batch Number: [ 36 ,  138 ]
	Train Batch Number: [ 37 ,  138 ]
	Train Batch Number: [ 38 ,  138 ]
	Train Batch Number: [ 39 ,  138 ]
	Train Batch Number: [ 40 ,  138 ]
	Train Batch Number: [ 41 ,  138 ]
	Train Batch Number: [ 42 ,  138 ]
	Train Batch Number: [ 43 ,  138 ]
	Train Batch Number: [ 44 ,  138 ]
	Train Batch Number: [ 45 ,  138 ]
	Train Batch Number: [ 46 ,  138 ]
	Train Batch Number: [ 47 ,  138 ]
	Train Batch Number: [ 48 ,  138 ]
	Train Batch Number: [ 49 ,  138 ]
	Train Batch Number: [ 50 ,  138 ]
	Train Batch Number: [ 51 ,  138 ]
	Train Batch Number: [ 52 ,  138 ]
	Train Batch Number: [ 53 ,  138 ]
	Train Batch Number:

	Train Batch Number: [ 113 ,  138 ]
	Train Batch Number: [ 114 ,  138 ]
	Train Batch Number: [ 115 ,  138 ]
	Train Batch Number: [ 116 ,  138 ]
	Train Batch Number: [ 117 ,  138 ]
	Train Batch Number: [ 118 ,  138 ]
	Train Batch Number: [ 119 ,  138 ]
	Train Batch Number: [ 120 ,  138 ]
	Train Batch Number: [ 121 ,  138 ]
	Train Batch Number: [ 122 ,  138 ]
	Train Batch Number: [ 123 ,  138 ]
	Train Batch Number: [ 124 ,  138 ]
	Train Batch Number: [ 125 ,  138 ]
	Train Batch Number: [ 126 ,  138 ]
	Train Batch Number: [ 127 ,  138 ]
	Train Batch Number: [ 128 ,  138 ]
	Train Batch Number: [ 129 ,  138 ]
	Train Batch Number: [ 130 ,  138 ]
	Train Batch Number: [ 131 ,  138 ]
	Train Batch Number: [ 132 ,  138 ]
	Train Batch Number: [ 133 ,  138 ]
	Train Batch Number: [ 134 ,  138 ]
	Train Batch Number: [ 135 ,  138 ]
	Train Batch Number: [ 136 ,  138 ]
	Train Batch Number: [ 137 ,  138 ]
	Validation Batch Number: [ 0 ,  5 ]
	Validation Batch Number: [ 1 ,  5 ]
	Validation Batch Number: 

	Train Batch Number: [ 53 ,  138 ]
	Train Batch Number: [ 54 ,  138 ]
	Train Batch Number: [ 55 ,  138 ]
	Train Batch Number: [ 56 ,  138 ]
	Train Batch Number: [ 57 ,  138 ]
	Train Batch Number: [ 58 ,  138 ]
	Train Batch Number: [ 59 ,  138 ]
	Train Batch Number: [ 60 ,  138 ]
	Train Batch Number: [ 61 ,  138 ]
	Train Batch Number: [ 62 ,  138 ]
	Train Batch Number: [ 63 ,  138 ]
	Train Batch Number: [ 64 ,  138 ]
	Train Batch Number: [ 65 ,  138 ]
	Train Batch Number: [ 66 ,  138 ]
	Train Batch Number: [ 67 ,  138 ]
	Train Batch Number: [ 68 ,  138 ]
	Train Batch Number: [ 69 ,  138 ]
	Train Batch Number: [ 70 ,  138 ]
	Train Batch Number: [ 71 ,  138 ]
	Train Batch Number: [ 72 ,  138 ]
	Train Batch Number: [ 73 ,  138 ]
	Train Batch Number: [ 74 ,  138 ]
	Train Batch Number: [ 75 ,  138 ]
	Train Batch Number: [ 76 ,  138 ]
	Train Batch Number: [ 77 ,  138 ]
	Train Batch Number: [ 78 ,  138 ]
	Train Batch Number: [ 79 ,  138 ]
	Train Batch Number: [ 80 ,  138 ]
	Train Batch Number:

...

	Train Batch Number: [ 108 ,  138 ]
	Train Batch Number: [ 109 ,  138 ]
	Train Batch Number: [ 110 ,  138 ]
	Train Batch Number: [ 111 ,  138 ]
	Train Batch Number: [ 112 ,  138 ]
	Train Batch Number: [ 113 ,  138 ]
	Train Batch Number: [ 114 ,  138 ]
	Train Batch Number: [ 115 ,  138 ]
	Train Batch Number: [ 116 ,  138 ]
	Train Batch Number: [ 117 ,  138 ]
	Train Batch Number: [ 118 ,  138 ]
	Train Batch Number: [ 119 ,  138 ]
	Train Batch Number: [ 120 ,  138 ]
	Train Batch Number: [ 121 ,  138 ]
	Train Batch Number: [ 122 ,  138 ]
	Train Batch Number: [ 123 ,  138 ]
	Train Batch Number: [ 124 ,  138 ]
	Train Batch Number: [ 125 ,  138 ]
	Train Batch Number: [ 126 ,  138 ]
	Train Batch Number: [ 127 ,  138 ]
	Train Batch Number: [ 128 ,  138 ]
	Train Batch Number: [ 129 ,  138 ]
	Train Batch Number: [ 130 ,  138 ]
	Train Batch Number: [ 131 ,  138 ]
	Train Batch Number: [ 132 ,  138 ]
	Train Batch Number: [ 133 ,  138 ]
	Train Batch Number: [ 134 ,  138 ]
	Train Batch Number: [ 135 ,

	Train Batch Number: [ 49 ,  138 ]
	Train Batch Number: [ 50 ,  138 ]
	Train Batch Number: [ 51 ,  138 ]
	Train Batch Number: [ 52 ,  138 ]
	Train Batch Number: [ 53 ,  138 ]
	Train Batch Number: [ 54 ,  138 ]
	Train Batch Number: [ 55 ,  138 ]
	Train Batch Number: [ 56 ,  138 ]
	Train Batch Number: [ 57 ,  138 ]
	Train Batch Number: [ 58 ,  138 ]
	Train Batch Number: [ 59 ,  138 ]
	Train Batch Number: [ 60 ,  138 ]
	Train Batch Number: [ 61 ,  138 ]
	Train Batch Number: [ 62 ,  138 ]
	Train Batch Number: [ 63 ,  138 ]
	Train Batch Number: [ 64 ,  138 ]
	Train Batch Number: [ 65 ,  138 ]
	Train Batch Number: [ 66 ,  138 ]
	Train Batch Number: [ 67 ,  138 ]
	Train Batch Number: [ 68 ,  138 ]
	Train Batch Number: [ 69 ,  138 ]
	Train Batch Number: [ 70 ,  138 ]
	Train Batch Number: [ 71 ,  138 ]
	Train Batch Number: [ 72 ,  138 ]
	Train Batch Number: [ 73 ,  138 ]
	Train Batch Number: [ 74 ,  138 ]
	Train Batch Number: [ 75 ,  138 ]
	Train Batch Number: [ 76 ,  138 ]
	Train Batch Number:

	Train Batch Number: [ 136 ,  138 ]
	Train Batch Number: [ 137 ,  138 ]
	Validation Batch Number: [ 0 ,  5 ]
	Validation Batch Number: [ 1 ,  5 ]
	Validation Batch Number: [ 2 ,  5 ]
	Validation Batch Number: [ 3 ,  5 ]
	Validation Batch Number: [ 4 ,  5 ]
Epoch: 16 	Training Loss: 2.7060598618742353 	Validation Loss: 1.1019454717636108
Epochs Number: 17
	Train Batch Number: [ 0 ,  138 ]
	Train Batch Number: [ 1 ,  138 ]
	Train Batch Number: [ 2 ,  138 ]
	Train Batch Number: [ 3 ,  138 ]
	Train Batch Number: [ 4 ,  138 ]
	Train Batch Number: [ 5 ,  138 ]
	Train Batch Number: [ 6 ,  138 ]
	Train Batch Number: [ 7 ,  138 ]
	Train Batch Number: [ 8 ,  138 ]
	Train Batch Number: [ 9 ,  138 ]
	Train Batch Number: [ 10 ,  138 ]
	Train Batch Number: [ 11 ,  138 ]
	Train Batch Number: [ 12 ,  138 ]
	Train Batch Number: [ 13 ,  138 ]
	Train Batch Number: [ 14 ,  138 ]
	Train Batch Number: [ 15 ,  138 ]
	Train Batch Number: [ 16 ,  138 ]
	Train Batch Number: [ 17 ,  138 ]
	Train Batch Number: [ 

	Train Batch Number: [ 78 ,  138 ]
	Train Batch Number: [ 79 ,  138 ]
	Train Batch Number: [ 80 ,  138 ]
	Train Batch Number: [ 81 ,  138 ]
	Train Batch Number: [ 82 ,  138 ]
	Train Batch Number: [ 83 ,  138 ]
	Train Batch Number: [ 84 ,  138 ]
	Train Batch Number: [ 85 ,  138 ]
	Train Batch Number: [ 86 ,  138 ]
	Train Batch Number: [ 87 ,  138 ]
	Train Batch Number: [ 88 ,  138 ]
	Train Batch Number: [ 89 ,  138 ]
	Train Batch Number: [ 90 ,  138 ]
	Train Batch Number: [ 91 ,  138 ]
	Train Batch Number: [ 92 ,  138 ]
	Train Batch Number: [ 93 ,  138 ]
	Train Batch Number: [ 94 ,  138 ]
	Train Batch Number: [ 95 ,  138 ]
	Train Batch Number: [ 96 ,  138 ]
	Train Batch Number: [ 97 ,  138 ]
	Train Batch Number: [ 98 ,  138 ]
	Train Batch Number: [ 99 ,  138 ]
	Train Batch Number: [ 100 ,  138 ]
	Train Batch Number: [ 101 ,  138 ]
	Train Batch Number: [ 102 ,  138 ]
	Train Batch Number: [ 103 ,  138 ]
	Train Batch Number: [ 104 ,  138 ]
	Train Batch Number: [ 105 ,  138 ]
	Train Batch N

	Train Batch Number: [ 19 ,  138 ]
	Train Batch Number: [ 20 ,  138 ]
	Train Batch Number: [ 21 ,  138 ]
	Train Batch Number: [ 22 ,  138 ]
	Train Batch Number: [ 23 ,  138 ]
	Train Batch Number: [ 24 ,  138 ]
	Train Batch Number: [ 25 ,  138 ]
	Train Batch Number: [ 26 ,  138 ]
	Train Batch Number: [ 27 ,  138 ]
	Train Batch Number: [ 28 ,  138 ]
	Train Batch Number: [ 29 ,  138 ]
	Train Batch Number: [ 30 ,  138 ]
	Train Batch Number: [ 31 ,  138 ]
	Train Batch Number: [ 32 ,  138 ]
	Train Batch Number: [ 33 ,  138 ]
	Train Batch Number: [ 34 ,  138 ]
	Train Batch Number: [ 35 ,  138 ]
	Train Batch Number: [ 36 ,  138 ]
	Train Batch Number: [ 37 ,  138 ]
	Train Batch Number: [ 38 ,  138 ]
	Train Batch Number: [ 39 ,  138 ]
	Train Batch Number: [ 40 ,  138 ]
	Train Batch Number: [ 41 ,  138 ]
	Train Batch Number: [ 42 ,  138 ]
	Train Batch Number: [ 43 ,  138 ]
	Train Batch Number: [ 44 ,  138 ]
	Train Batch Number: [ 45 ,  138 ]
	Train Batch Number: [ 46 ,  138 ]
	Train Batch Number:

In [7]:
### begin test loop
test(loaders, Classifier_net, mod
     el_criterion, accelerator, verbose=True)

Testing starts...

Test Loss: 1.097545774359452

Test Accuracy: 28.0%, (168/600)
Threat Score: 0.07449900433946845%
Testing ends...

